In [ ]:
# reads an NWB file and prints its contents
from pynwb import NWBHDF5IO
import os

file_name = "example_file.nwb"
folder_path = "Test"

# Create full file path
full_file_path = os.path.join(folder_path, file_name)

with NWBHDF5IO(full_file_path, "r") as io:
    nwbfile = io.read()
    print(nwbfile)



root pynwb.file.NWBFile at 0x2234826980752
Fields:
  acquisition: {
    array_data <class 'pynwb.base.TimeSeries'>,
    image_data <class 'pynwb.base.TimeSeries'>,
    int_data <class 'pynwb.base.TimeSeries'>,
    list_data <class 'pynwb.base.TimeSeries'>,
    string_data <class 'pynwb.base.TimeSeries'>
  }
  file_create_date: [datetime.datetime(2025, 1, 16, 11, 52, 40, 824430, tzinfo=tzoffset(None, 7200))]
  identifier: example-id
  session_description: Python object storage
  session_start_time: 2025-01-16 09:52:40.824430+00:00
  timestamps_reference_time: 2025-01-16 09:52:40.824430+00:00



In [ ]:
# create a new NWB file
# create nwb file and save initial skeleton to disk


"""
Create 
TimeSeries - units spikes times
ImageSeries - unique images presented using the projector
DynamicTable - time of image presentation and link to index of the image presented

acquisition: raw data acquired from the experimental subject
   units: spike times of individual neurons
   stimulus: time series data that represents a optogenetic stimulus time series

"""
from pynwb import NWBFile, NWBHDF5IO
from pynwb.image import GrayscaleImage, Images
from pynwb.core import DynamicTable
from pynwb.file import Subject

from datetime import datetime, timezone
import numpy as np

culture_number = 1 # unique identifier for the culture
culture_DIV = 14 # days in vitro of the culture
culture_id = f'culture_{culture_number}_DIV{culture_DIV}'

subject = Subject(

    subject_id=culture_id,
    description="Primary cortical neuron culture from E18 rat embryos",
    species="Rattus norvegicus",
    genotype="WT",
    age="E18",  # Embryonic day 18 at dissection
    plating_date = datetime(2025, 3, 12),
    strain="c57BL/6"
    )


# Create NWB file
nwbfile = NWBFile(
    session_description='Optogenetic stimulation with static image stimuli',
    identifier='OPTO_IMG_SESSION_STATIC_ARRAY',
    session_start_time=datetime.now(timezone.utc)
)

# --- Properly store static images using Images container ---
unique_images_data = np.random.randint(0, 256, (11, 256, 256), dtype=np.uint8)

# Convert numpy arrays into NWB GrayscaleImage objects
image_objs = []
for idx, img_data in enumerate(unique_images_data):
    img = GrayscaleImage(name=f'img_{idx}', data=img_data, description=f'Opto stim image {idx}')
    image_objs.append(img)

# Store the GrayscaleImages in an Images container
image_container = Images(
    name='stimulus_image_library',
    images=image_objs,
    description='Static image library used for optogenetic stimulation'
)

# Explicitly add the static images to the stimulus group
nwbfile.add_stimulus(image_container)
#nwbfile.stimulus['stimulus_image_library'] = image_container

# processing/stimulus_presentation____________________________________________________________________________________
# --- Create a DynamicTable referencing the image indices ---
presentation_table = DynamicTable(
    name='image_time_index',
    description='Table recording presentation times of indiced images referencing the static image library'
)
presentation_table.add_column('start_time', 'Time at which image was presented')
presentation_table.add_column('image_index', 'Index of the presented image')

# Populate the presentation table
for i in range(100):
    img_idx = np.random.randint(0, 10)
    start_time = i * 0.5
    presentation_table.add_row(start_time=start_time, image_index=img_idx)

# Store the presentation metadata clearly in the processing module
proc_module = nwbfile.create_processing_module(
    name='stimulus_index_time',
    description='Stimulus timing data referencing static stimulus images'
)
proc_module.add_data_interface(presentation_table)

# Write NWB file
with NWBHDF5IO('optical_stim_session_static_array2.nwb', 'w') as io:
    io.write(nwbfile)



In [20]:
# open the above NWB file
#"G:\My Drive\Research\Projects\Theory of cortical mind\Object representation\Software\Python\QT_GUI\Develope\optical_stim_session_static_array.nwb"
from pynwb import NWBHDF5IO
import os


from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
import h5py

file_name = "optical_stim_session_static_array2.nwb"
folder_path = "G:\My Drive\Research\Projects\Theory of cortical mind\Object representation\Software\Python\QT_GUI\Develope"
# import the file
full_file_path = os.path.join(folder_path, file_name)


with h5py.File(full_file_path, 'r') as f:
    # This function will be called for every object in the file.
    def print_group(name, obj):
        if isinstance(obj, h5py.Group):
            print(name)

    f.visititems(print_group)


with NWBHDF5IO(full_file_path, 'r') as io:
    nwbfile = io.read()

    # Access the presentation_data processing module
    proc_module = nwbfile.processing['stimulus_index_time']

    # Access your dynamic table within the processing module
    presentation_table = proc_module['image_time_index']

    # Convert to DataFrame for easy viewing
    df = presentation_table.to_dataframe()
    print(df.head())
    # number of samples in the table
    print("We have ",len(df), " samples in the table")




acquisition
analysis
general
processing
processing/stimulus_index_time
processing/stimulus_index_time/image_time_index
specifications
specifications/core
specifications/core/2.7.0
specifications/hdmf-common
specifications/hdmf-common/1.8.0
specifications/hdmf-experimental
specifications/hdmf-experimental/0.5.0
specifications/ndx-grayscalevolume
specifications/ndx-grayscalevolume/0.0.2
specifications/ndx-icephys-meta
specifications/ndx-icephys-meta/0.1.0
specifications/ndx-spectrum
specifications/ndx-spectrum/0.2.2
stimulus
stimulus/presentation
stimulus/presentation/stimulus_image_library
stimulus/templates
    start_time  image_index
id                         
0          0.0            0
1          0.5            6
2          1.0            9
3          1.5            1
4          2.0            5
We have  100  samples in the table


In [ ]:
# use the nwbwidgets to visualize the data
from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO

file_name = "optical_stim_session_static_array2.nwb"
#folder_path = "G:\My Drive\Research\Projects\Theory of cortical mind\Object representation\Software\Python\QT_GUI\Develope"

nwb = NWBHDF5IO(file_name, 'r').read()


: 

In [17]:
nwb2widget(nwb)

In [4]:
# load the nwb file and extract the variables back to the original format

from pynwb import NWBHDF5IO
import numpy as np


def extract_group_data(group, group_dict):
    if hasattr(group, 'keys'):
        for key in group.keys():
            value = group[key]
            if hasattr(value, 'data'):
                group_dict[key] = np.array(value.data)
            else:
                subgroup_dict = {}
                extract_group_data(value, subgroup_dict)
                group_dict[key] = subgroup_dict
    elif hasattr(group, '__iter__') and not isinstance(group, str):
        for idx, item in enumerate(group):
            subgroup_dict = {}
            extract_group_data(item, subgroup_dict)
            group_dict[f'item_{idx}'] = subgroup_dict
    else:
        group_dict['value'] = str(group)


def extract_attributes(obj):
    attrs = {}
    for attr in dir(obj):
        if not attr.startswith('_') and not callable(getattr(obj, attr)):
            attrs[attr] = getattr(obj, attr)
    return attrs


def extract_nwb_file(filepath):
    extracted_data = {}
    with NWBHDF5IO(filepath, 'r') as io:
        nwbfile = io.read()

        # Extract subject information dynamically
        extracted_data['subject'] = extract_attributes(nwbfile.subject)

        # Extract stimulus data
        extracted_data['stimulus'] = {}
        extract_group_data(nwbfile.stimulus, extracted_data['stimulus'])

        # Extract processing modules
        extracted_data['processing'] = {}
        for mod_name in nwbfile.processing:
            mod = nwbfile.processing[mod_name]
            extracted_data['processing'][mod_name] = {}
            extract_group_data(mod.data_interfaces, extracted_data['processing'][mod_name])

        # Extract acquisition data
        extracted_data['acquisition'] = {}
        extract_group_data(nwbfile.acquisition, extracted_data['acquisition'])

    return extracted_data



# Example usage:
nwb_data = extract_nwb_file('G:\My Drive\Research\Projects\Theory of cortical mind\Object representation\Software\Python\QT_GUI\Develope\optical_stim_session_static_array2.nwb')

# Inspect extracted data
for key, val in nwb_data.items():
    print(f'{key}: {type(val)}')

    

subject: <class 'dict'>
stimulus: <class 'dict'>
processing: <class 'dict'>
acquisition: <class 'dict'>


In [1]:
import os
from datetime import datetime
from zoneinfo import ZoneInfo
from pathlib import Path
from neuroconv.datainterfaces import MaxOneRecordingInterface

# Updated file path with the correct location and file name
#file_path = r"L:\gogomatlab\MATLAB_Exp2_ReadFiles\MaxWell\0YonData\Trace_20250122_10_56_37.raw.h5"
file_path = r"L:\gogomatlab\MATLAB_Exp2_ReadFiles\MaxWell\0YonData\sync_culture_42DIV.raw.h5"

interface = MaxOneRecordingInterface(file_path=file_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()

# For data provenance we add the time zone information to the conversion
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=ZoneInfo("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

# Save the NWB file in the specified folder
nwbfile_path = "NWB convert"
interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)



NotImplementedError: The MaxOneRecordingInterface has not yet been implemented for systems other than Linux.

In [1]:

protocols_number = 1 # unique identifier for the protocol
file_name = f"protocol_{protocols_number}.pkl"
print(file_name)


protocol_1.pkl
